### Utilizando a base de dados archive.ics.uci.edu/ml/datasets/Car+Evaluation:

In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('car.csv')
data.head()

,buying,maint,doors,persons,lug_boot,safety,class
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


### 1. (10 pontos) Defina o classificador cego com o menor erro de classificação possível. 

##### calcular a probabilidade a priori de cada classe

In [9]:
tot  = data['class'].size
tot

1728

In [10]:
cont = data['class'].value_counts()
cont

unacc    1210
acc       384
good       69
vgood      65
Name: class, dtype: int64

In [16]:
sum = 0
for i in cont:
    prob = i/tot
    print(prob)
    sum = sum + prob
    
sum

0.7002314814814815
0.2222222222222222
0.03993055555555555
0.03761574074074074


1.0

##### Erro maximo para cada classe

In [18]:
#unacc
1 - (cont[0]/tot)

0.2997685185185185

In [21]:
#acc
1 - (cont[1]/tot)

0.7777777777777778

In [22]:
#good
1 - (cont[2]/tot)

0.9600694444444444

In [25]:
#vgood
1 - (cont[3]/tot)

0.9623842592592593

<font color="yellow">Resposta: O classificador cego com o menor erro de classificação possível é unacc (Erro máximo = 0.2997685185185185)</font>

### 2. (18 pontos) Calcule as probablidades a priori para cada variável (para cada possível valor de cada variável).

In [73]:
for x in data.drop(['class'], axis=1):
    unique_values = np.unique(data[x].values)
    print("\n" + x + "\n")
    for y in unique_values:
        print(y, str(data[x].value_counts()[y]) + "/" + str(data[x].size))


buying

high 432/1728
low 432/1728
med 432/1728
vhigh 432/1728

maint

high 432/1728
low 432/1728
med 432/1728
vhigh 432/1728

doors

2 432/1728
3 432/1728
4 432/1728
5more 432/1728

persons

2 576/1728
4 576/1728
more 576/1728

lug_boot

big 576/1728
med 576/1728
small 576/1728

safety

high 576/1728
low 576/1728
med 576/1728


### 3. (18 pontos) Calcule a probabilidade de cada valor de cada variável dado a classe (posteriori).

In [70]:
class_values = np.unique(data['class'].values)

for x in data.drop(["class"], axis=1):
    unique_values = np.unique(data[x].values)
    print("\n" + x + "\n")
    for y in unique_values:
        for i in class_values:
            try:
                print(y + " | " + i, str(data.groupby(x)['class'].value_counts()[y][i]) + "/" + str(data['class'].value_counts()[i]))
            except:
                next




buying

high | acc 108/384
high | unacc 324/1210
low | acc 89/384
low | good 46/69
low | unacc 258/1210
low | vgood 39/65
med | acc 115/384
med | good 23/69
med | unacc 268/1210
med | vgood 26/65
vhigh | acc 72/384
vhigh | unacc 360/1210

maint

high | acc 105/384
high | unacc 314/1210
high | vgood 13/65
low | acc 92/384
low | good 46/69
low | unacc 268/1210
low | vgood 26/65
med | acc 115/384
med | good 23/69
med | unacc 268/1210
med | vgood 26/65
vhigh | acc 72/384
vhigh | unacc 360/1210

doors

2 | acc 81/384
2 | good 15/69
2 | unacc 326/1210
2 | vgood 10/65
3 | acc 99/384
3 | good 18/69
3 | unacc 300/1210
3 | vgood 15/65
4 | acc 102/384
4 | good 18/69
4 | unacc 292/1210
4 | vgood 20/65
5more | acc 102/384
5more | good 18/69
5more | unacc 292/1210
5more | vgood 20/65

persons

2 | unacc 576/1210
4 | acc 198/384
4 | good 36/69
4 | unacc 312/1210
4 | vgood 30/65
more | acc 186/384
more | good 33/69
more | unacc 322/1210
more | vgood 35/65

lug_boot

big | acc 144/384
big | good 24/69

### 4. (18 pontos) Implemente o classificador Naive Bayes.

In [5]:
class Naive_bayes:
                        #Retorna a probabilidade a priori de cada classe em um DataFrame
    
    #data = DataFrame
    #y = nome da coluna de classes
    def prob_y(self, data, y):
        prob = {}
        for i in data[y].unique().tolist():
            a = data[y].value_counts()[i]
            b = data[y].count()  
            prob[i] = a/b

        self.prob_y = pd.DataFrame([prob])


                                        #Calcula P(x|c)

    #col  = nome da coluna onde se encontra x
    #x    = valor da coluna 'col' no exemplo
    #c    = valor da coluna 'y' no DataFrame (valor da classe)
    #y    = nome da coluna de classes
    #data = DataFrame
    def prob_x_c(self, x, c, y, col, data):
        #Ocorrencias de c em y
        ci = data.loc[data[y] == c].shape[0]
        #Interseção de col = x e y = c 
        d = data.loc[(data[col] == x) & (data[y] == c)].shape[0]
        #Tamanho 
        nx = len(data[y].unique())
        return (d + 1)/(ci + nx)


                            #Função discriminante para o classificador bayesiano

    #x = vetor de caracteristicas (DataFrame)
    #c = valor de y no DataFrame
    #y = nome da coluna de classes no DataFrame
    #data = DataFrame
    def funcao_discriminante(self, x, c):
        prob = self.prob_y[c].values[0]

        for i in x.columns:
            name = i+'_'+x[i].values[0]
            line = self.nb_table.loc[self.nb_table['name'] == name]
            prob = prob * line[c].values[0]
        
        return prob
        

                            #Função de treinamento para o classificador

    #deve retornar a tabela com as probabilidades:
    #+----------------------------------------------------------------------+
    #       name        unacc           acc         vgood	        good
    #   buying_low     P(bl|unacc)     P(bl|acc)   P(bl|vgood)   P(bl|good)
    #   buying_vhigh   P(bv|unacc)     P(bv|acc)   P(bv|vgood)   P(bv|good)
    #      ...            ...             ...         ...           ...
    #   safety_high    P(sh|unacc)     P(sh|acc)   P(sh|vgood)   P(sh|good)
    #+----------------------------------------------------------------------+
    #data = DataFrame
    #y = nome da coluna de classes
    def train(self, data, y):
        prob = {'name':[], 'unacc':[], 'acc':[], 'vgood':[], 'good':[]}

        #Calcula a probabilidade dos valores de y no dataFrame
        self.prob_y(data, y)
        
        #para todos as colunas de data
        for col in data.drop([y], axis=1).columns:
            #para todos os valores possíveis da coluna i
            for x in data[col].unique().tolist():
                prob['name'].append(col+'_'+x)
                
                #para todos os valores possiveis da classe 
                for c in data[y].unique().tolist():
                    prob[c].append(self.prob_x_c(x, c, y, col, data))
        self.nb_table =  pd.DataFrame(prob)


                                        #Função de classificação

    #retorna o vetor de classificação das linhas de data
    #data = dataFrame dos vetores de caracteristicas
    def predict(self, data):
        y = []

        #para cada linha de data
        for i, j in data.iterrows():
            result = ''                 #classificação atual
            max = 0                     #resultado da probabilidade de P(xi|Ci), onde Ci = result
            
            #para cada valor possivel de y
            for k in self.nb_table.drop(['name'], axis=1).columns.to_list(): 
                #discriminante
                fd = self.funcao_discriminante(pd.DataFrame([j]), k)
                #caso descriminante maior que o max, substitui result por k, e max por discriminante
                if (fd > max):
                    max = fd
                    result = k
            
            #Adiciona a classificação para o exemplo j no vetor de classificação y
            y.append(result)

        return y


                                #Função para calcular a acuracia do classificador

    #retorna a acuraia e o erro do classificador
    #predict = valor predito pelo classificador
    #true_values = os valores reais das classes do conjunto de teste
    def score(self, predict, true_values):
        accuracy, erro = 0, 0
        correct, incorrect = 0, 0

        for i in range(len(predict)):
            if (predict[i] == true_values[i]):
                correct = correct + 1
            else:
                incorrect = incorrect + 1

        accuracy = correct/len(predict)
        erro = incorrect/len(predict)
        return accuracy, erro

### 5. (18 pontos) Utilizando sua implementação e metade dos exemplos de cada classe como conjunto de treino e o resante como conjunto de teste, avalie o classificador Naive Bayes.

In [2]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(data.drop(['class'], axis=1), data['class'], test_size=0.5)

In [6]:
x = x_train.copy()
x['class'] = y_train.copy()
x.head()

,buying,maint,doors,persons,lug_boot,safety,class
1162,med,med,5more,2,small,med,unacc
317,vhigh,med,5more,more,small,high,acc
956,med,vhigh,5more,4,small,high,acc
1039,med,high,4,4,med,med,acc
1366,low,vhigh,4,4,big,med,acc


<font color='#ebaf09'>Testando a saída do treino</font>

In [7]:
nb = Naive_bayes()
nb.train(x, 'class')
nb.nb_table

,name,unacc,acc,vgood,good
0,buying_med,0.235000,0.312796,0.371429,0.382353
1,buying_vhigh,0.276667,0.170616,0.028571,0.029412
2,buying_low,0.218333,0.246445,0.571429,0.558824
3,buying_high,0.270000,0.270142,0.028571,0.029412
4,maint_med,0.220000,0.289100,0.428571,0.264706
5,maint_vhigh,0.301667,0.208531,0.028571,0.029412
6,maint_high,0.263333,0.289100,0.171429,0.029412
7,maint_low,0.215000,0.213270,0.371429,0.676471
8,doors_5more,0.243333,0.274882,0.228571,0.352941
9,doors_4,0.240000,0.270142,0.314286,0.205882


<font color='#ebaf09'>Classificando o conjunto de teste</font>

In [8]:
result = nb.predict(x_test)
result

['unacc',
 'unacc',
 'acc',
 'acc',
 'unacc',
 'unacc',
 'unacc',
 'acc',
 'acc',
 'unacc',
 'unacc',
 'acc',
 'acc',
 'acc',
 'unacc',
 'acc',
 'unacc',
 'unacc',
 'unacc',
 'unacc',
 'unacc',
 'acc',
 'acc',
 'acc',
 'unacc',
 'acc',
 'unacc',
 'acc',
 'acc',
 'acc',
 'unacc',
 'unacc',
 'unacc',
 'acc',
 'unacc',
 'unacc',
 'acc',
 'unacc',
 'unacc',
 'unacc',
 'unacc',
 'unacc',
 'unacc',
 'unacc',
 'acc',
 'unacc',
 'unacc',
 'unacc',
 'unacc',
 'acc',
 'acc',
 'unacc',
 'acc',
 'unacc',
 'unacc',
 'unacc',
 'acc',
 'unacc',
 'unacc',
 'unacc',
 'unacc',
 'acc',
 'unacc',
 'unacc',
 'acc',
 'unacc',
 'unacc',
 'unacc',
 'unacc',
 'unacc',
 'unacc',
 'acc',
 'acc',
 'unacc',
 'unacc',
 'acc',
 'unacc',
 'acc',
 'acc',
 'unacc',
 'unacc',
 'acc',
 'acc',
 'unacc',
 'unacc',
 'acc',
 'unacc',
 'unacc',
 'unacc',
 'acc',
 'unacc',
 'unacc',
 'unacc',
 'unacc',
 'unacc',
 'unacc',
 'acc',
 'unacc',
 'unacc',
 'unacc',
 'unacc',
 'unacc',
 'unacc',
 'unacc',
 'unacc',
 'unacc',
 'unacc'

<font color='#ebaf09'>Verificando a accuracia e o erro do classificador</font>

In [9]:
nb.score(result, y_test.values.tolist())

(0.8541666666666666, 0.14583333333333334)

### 6. (18 pontos) Faça o mesmo da questão anterior utilizando uma implementação de biblioteca com configuração semelhente ao classificador solicitado na questão anterior.

In [22]:
map = [
        {'col':'buying', 
        'mapping':{'low': 0, 'med':1, 'high':2, 'vhigh':3}},
        {'col':'maint',
        'mapping':{'low': 0, 'med':1, 'high':2, 'vhigh':3}},
        {'col':'doors',
        'mapping':{'2': 0, '3':1, '4':2, '5more':3}},
        {'col':'persons',
        'mapping':{'2': 0, '4':1, 'more':2}},
        {'col':'lug_boot',
        'mapping':{'small': 0, 'med':1, 'big':2}},
        {'col':'safety',
        'mapping':{'low': 0, 'med':1, 'high':2}},
        {'col':'class',
        'mapping':{'unacc': 0, 'acc':1, 'vgood':2, 'good':3}}
        ]

In [32]:
from category_encoders import OrdinalEncoder

oe = OrdinalEncoder(mapping=map)
x_train['class'] = y_train

In [ ]:
x_train_enc = oe.fit_transform(x_train)
x_test['class'] = y_test
x_test_enc = oe.fit_transform(x_test)

y_train_enc = x_train_enc['class']
y_test_enc = x_test_enc['class']

x_train_enc = x_train_enc.drop(['class'], axis=1)
x_test_enc = x_test_enc.drop(['class'], axis=1)

In [37]:
from sklearn.naive_bayes import CategoricalNB

cnb = CategoricalNB()
cnb.fit(x_train_enc, y_train_enc)
predict = cnb.predict(x_test_enc)

In [71]:
from sklearn import metrics

metrics.accuracy_score(y_test_enc, predict)

0.8530092592592593